# Ceneo Scraper

# Biblioteki

In [16]:
import os
import json
import requests 
from bs4 import BeautifulSoup


# Funkcja do

In [23]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()   

In [18]:
selector= {
    "opinion_id" : (None, "data-entry-id"),
    "author" : ("span.user-post__author-name",),
    "recomendation" : ("span.user-post__author-recomendation > em",),
    "rating" :  ("span.user-post__score-count",),
    "content" : ("div.user-post__text",),
    "pros" : ("div.review-feature_title-positives ~ div.review-features_item", None, True),
    "cons" : ("div.review-feature_title-negatives ~ div.review-features_item", None, True),
    "useful" : ("button.vote-yes > span",),
    "useless" : ( "button.vote-no > span",),
    "publish_date" : ("span.user-post__published > time:nth-child(1)","datetime"),
    "purshare_date" : ("span.user-post__published > time:nth-child(2)", "datetime"),
    }

# Wysyłanie do servera żądania dostępu do zasobu 

In [19]:
product_id = "94103927"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"


In [24]:
all_opinions = []
while (url):
    response = requests.get(url)
    response.status_code
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion in opinions:
        try:
            single_opinion = {
                key: extract(opinion, *value)
                    for key, value in selector.items()
            }
            all_opinions.append(single_opinion)    
        except (TypeError, AttributeError):
            pass    
    try:
        url = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href") 
    except TypeError:
        url = None       

## Ekstrakcja składowych pojedynczej opinii 
|Składowa|Selector|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"] |opinion_id|
|autor|span.user-post_author-name |author|
|rekomendacja| span.user-post_author-recomendation >em |recomendation||
|gwiazdki| span.user-post_score-count |rating|
|treść| div.user-post_text |content|
|lista zalet| div.review-feature_title-positives ~ div.review-features_item|pros|
|lista wad| div.review-feature_title-negatives ~ div.review-features_item|cons|
|dla ilu przydatna| button.votes-yes > span |useful|
|dla ilu nieprzydatna| button.votes-no > span |useless|
|data wystawienia|span.user-post_published > time:nth-child(1)['datetime'] |publish_date|
|data zakupu| span.user-post_published > time:nth-child(2)['datetime'] |purshare_date|

In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open("opinions/{product_id}.json", "w", encoding="UTF-8") as jsonfile:    
    json.dump(all_opinions, jsonfile, indent = 4, ensure_ascii=False)

In [ ]:
print(len(all_opinions))

38
